<a href="https://colab.research.google.com/github/hanzla-khalid98/challenge_AN2DL_2025/blob/main/inference_hybrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Challenge 1**

---


In [1]:
from google.colab import drive
drive.mount('/content/drive')   # Optional, only if you want to save output to Drive


Mounted at /content/drive


In [2]:
!git clone https://github.com/hanzla-khalid98/challenge_AN2DL_2025.git

Cloning into 'challenge_AN2DL_2025'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 112 (delta 52), reused 67 (delta 20), pack-reused 0 (from 0)
Receiving objects: 100% (112/112), 17.12 MiB | 14.43 MiB/s, done.
Resolving deltas: 100% (52/52), done.
Filtering content: 100% (4/4), 186.05 MiB | 30.52 MiB/s, done.


In [3]:
import sys
sys.path.append('/hanzla-khalid98/challenge_AN2DL_2025/src')

## ⚙️ **Libraries Import**

In [4]:
# Set seed for reproducibility
SEED = 42

# Import necessary libraries
import os

# Set environment variables before importing modules
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['MPLCONFIGDIR'] = os.getcwd() + '/configs/'

# Suppress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=Warning)

# Import necessary modules
import logging
import random
import numpy as np

# Set seeds for random number generators in NumPy and Python
np.random.seed(SEED)
random.seed(SEED)

# Import PyTorch
import torch
torch.manual_seed(SEED)
from torch import nn
# from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import TensorDataset, DataLoader
logs_dir = "tensorboard"
!pkill -f tensorboard
%load_ext tensorboard
!mkdir -p models

if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device("cpu")

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {device}")

# Import other libraries
import copy
import shutil
from itertools import product
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Configure plot display settings
sns.set(font_scale=1.4)
sns.set_style('white')
plt.rc('font', size=14)
%matplotlib inline

PyTorch version: 2.8.0+cu126
Device: cpu


In [5]:
hparams_file = "/content/hybrid_hparams.pt"

model_file = "/content/hybrid_lstm_gru_model.pt"

BATCH_SIZE = 64

## ⏳ **Data Loading**

In [10]:
# inference dataset loading
data_inference = torch.load("/content/dataset_inference.pt")

inference_ds = TensorDataset(data_inference["list_sample_idx"], data_inference["X_inference"])


In [19]:
data_inference.keys()

dict_keys(['X_inference', 'list_sample_idx'])

In [20]:
def make_loader(ds, batch_size, shuffle, drop_last):
    # Determine optimal number of worker processes for data loading
    cpu_cores = os.cpu_count() or 2
    num_workers = max(2, min(4, cpu_cores))

    # Create DataLoader with performance optimizations
    return DataLoader(
        ds,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers,
        pin_memory=True,  # Faster GPU transfer
        pin_memory_device="cuda" if torch.cuda.is_available() else "",
        prefetch_factor=4,  # Load 4 batches ahead
    )
inference_loader = make_loader(inference_ds, batch_size=64, shuffle=True, drop_last=False)


# Model Recreation

In [21]:
class RecurrentClassifier(nn.Module):
    """
    Generic RNN classifier (RNN, LSTM, GRU).
    Uses the last hidden state for classification.
    """
    def __init__(
            self,
            input_size,
            hidden_size,
            num_layers,
            num_classes,
            rnn_type='GRU',        # 'RNN', 'LSTM', or 'GRU'
            bidirectional=False,
            dropout_rate=0.2
            ):
        super().__init__()

        self.rnn_type = rnn_type
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.bidirectional = bidirectional

        # Map string name to PyTorch RNN class
        rnn_map = {
            'RNN': nn.RNN,
            'LSTM': nn.LSTM,
            'GRU': nn.GRU
        }

        if rnn_type not in rnn_map:
            raise ValueError("rnn_type must be 'RNN', 'LSTM', or 'GRU'")

        rnn_module = rnn_map[rnn_type]

        # Dropout is only applied between layers (if num_layers > 1)
        dropout_val = dropout_rate if num_layers > 1 else 0

        # Create the recurrent layer
        self.rnn = rnn_module(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,       # Input shape: (batch, seq_len, features)
            bidirectional=bidirectional,
            dropout=dropout_val
        )

        # Calculate input size for the final classifier
        if self.bidirectional:
            classifier_input_size = hidden_size * 2 # Concat fwd + bwd
        else:
            classifier_input_size = hidden_size

        # Final classification layer
        self.classifier = nn.Linear(classifier_input_size, num_classes)

    def forward(self, x):
        """
        x shape: (batch_size, seq_length, input_size)
        """

        # rnn_out shape: (batch_size, seq_len, hidden_size * num_directions)
        rnn_out, hidden = self.rnn(x)

        # LSTM returns (h_n, c_n), we only need h_n
        if self.rnn_type == 'LSTM':
            hidden = hidden[0]

        # hidden shape: (num_layers * num_directions, batch_size, hidden_size)

        if self.bidirectional:
            # Reshape to (num_layers, 2, batch_size, hidden_size)
            hidden = hidden.view(self.num_layers, 2, -1, self.hidden_size)

            # Concat last fwd (hidden[-1, 0, ...]) and bwd (hidden[-1, 1, ...])
            # Final shape: (batch_size, hidden_size * 2)
            hidden_to_classify = torch.cat([hidden[-1, 0, :, :], hidden[-1, 1, :, :]], dim=1)
        else:
            # Take the last layer's hidden state
            # Final shape: (batch_size, hidden_size)
            hidden_to_classify = hidden[-1]

        # Get logits
        logits = self.classifier(hidden_to_classify)
        return logits


In [22]:
class HybridLSTMGRUClassifier(nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout_rate=0.2):
        super().__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout_rate if num_layers > 1 else 0
        )

        self.gru = nn.GRU(
            input_size=hidden_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout_rate if num_layers > 1 else 0
        )

        self.classifier = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):

        lstm_out, (h_lstm, _) = self.lstm(x)
        h_lstm_last = h_lstm[-1]

        gru_out, h_gru = self.gru(lstm_out)
        h_gru_last = h_gru[-1]

        combined = torch.cat((h_lstm_last, h_gru_last), dim=1)

        logits = self.classifier(combined)
        return logits

In [24]:
hparams = torch.load(hparams_file)

model = HybridLSTMGRUClassifier(
    input_size=hparams["input_size"], # Pass the number of features
    hidden_size=hparams["hidden_size"],
    num_layers=hparams["num_layers"],
    num_classes=hparams["num_classes"],
    dropout_rate=hparams["dropout_rate"]
    ).to(device)

model.load_state_dict(torch.load(model_file, map_location=device))

<All keys matched successfully>

## 🔎 **Predictions**

In [25]:
# Collect predictions and ground truth labels
list_preds = []
list_idx = []
with torch.no_grad():  # Disable gradient computation for inference
    for idx, xb in inference_loader:
        xb = xb.to(device) # Unpack the tensor from the list

        # Forward pass: get model predictions
        logits = model(xb)
        preds = logits.argmax(dim=1).cpu().numpy()

        # Store batch results
        list_preds.append(preds)
        list_idx.append(idx.numpy())

# Combine all batches into single arrays
list_preds = np.concatenate(list_preds)
list_idx = np.concatenate(list_idx)

# Create a DataFrame for the predictions
df_preds = pd.DataFrame({"sample_index": list_idx, "label": list_preds})

In [26]:
print(df_preds.shape)
df_preds.head()

(21184, 2)


,sample_index,label
0,846,0
1,311,0
2,899,0
3,1192,0
4,1224,0


In [27]:
# Group by sample index and get the mode of the labels.
# If there are multiple modes, take the first one.
df_preds = df_preds.groupby(['sample_index'])['label'].agg(lambda x: pd.Series.mode(x)[0]).reset_index()

In [28]:
df_preds.shape

(1324, 2)

In [29]:
# mapping labels 0, 1 and 2 to no_pain, low_pain and high_pain
mapping = {0: 'no_pain', 1: 'low_pain', 2: 'high_pain'}
df_preds['label'] = df_preds['label'].map(mapping)
df_preds["sample_index"] = df_preds["sample_index"].apply(lambda x: f"{int(x):03d}")

In [30]:
df_preds

,sample_index,label
0,000,no_pain
1,001,no_pain
2,002,no_pain
3,003,no_pain
4,004,no_pain
...,...,...
1319,1319,no_pain
1320,1320,no_pain
1321,1321,no_pain
1322,1322,no_pain


In [31]:
# save df_preds in csv without index, with header and with sample index with format 000, 001
df_preds.to_csv('predictions.csv', index=False, header=True)